In [1]:
import os
import numpy as np
from PIL import Image , ImageOps
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import h5py
from keras.models import load_model
from sklearn.model_selection import train_test_split
from __future__ import print_function
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
%matplotlib inline



np.random.seed(123)

Using TensorFlow backend.


In [2]:
def model_train(dir_img_True = '../data pre processing/croped data/malaria/',dir_img_False = '../data pre processing/croped data/no malaria/', weights_file = './model_weights.h5' ,   batch_size = 128 , num_classes = 2 , epochs = 150 , img_rows = 40 , img_cols = 40 ):

    malaria = []
    no_malaria = []
    dataset_x = []
    dataset_y = []
    # Open images into an array and preprocess them
    # Load malaria
    for file in os.listdir(dir_img_True):

        if file.endswith('.jpg'):
            #pre process images
            img = mpimg.pil_to_array(Image.open(dir_img_True+file))
            if img.shape[0] == 40 and img.shape[1] == 40:
                malaria.append([img[:,:,0].reshape(40,40,-1)])
    
    # Load no malaria
    for file in os.listdir(dir_img_False):
        if file.endswith('.jpg'):
            #pre process images
            img = mpimg.pil_to_array(Image.open(dir_img_False+file))
            if img.shape[0] == 40 and img.shape[1] == 40:
                no_malaria.append([img[:,:,0].reshape(40,40,-1)])   
    
    # Image preprocessing to array
    malaria = np.asarray(malaria)
    no_malaria = np.asarray(no_malaria)
    malaria = malaria.reshape(malaria.shape[0],40,40,-1)
    no_malaria = no_malaria.reshape(no_malaria.shape[0],40,40,-1)
    malaria_y = np.ones((malaria.shape[0],1)).reshape(malaria.shape[0],)
    no_malaria_y = np.zeros((no_malaria.shape[0],1)).reshape(no_malaria.shape[0],)
    dataset_x = np.concatenate((malaria, no_malaria), axis=0)
    dataset_y = np.concatenate((malaria_y, no_malaria_y), axis=0)

    # Split into test and train
    x_train, x_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.33, random_state=42)

    #actual train on CNN

    input_shape = (img_rows, img_cols, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    # Definition of the CNN
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))  # Anterior 128
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    
    # Training
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
                                                
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])                                            


    # save weights 
    model.save_weights(weights_file+'.h5')
    json_string = model.to_json()
    with open(weights_file+'.json', 'w') as jsonfile:
        jsonfile.write(json_string)

In [ ]:
model_train(weights_file = './model_weights-test',batch_size = 128 , epochs = 150 )


Train on 8524 samples, validate on 4199 samples
Epoch 1/150
8524/8524 [==============================] - 48s - loss: 0.7019 - acc: 0.5614 - val_loss: 0.6623 - val_acc: 0.6159
Epoch 2/150
8524/8524 [==============================] - 62s - loss: 0.6033 - acc: 0.6479 - val_loss: 0.4959 - val_acc: 0.7511
Epoch 3/150
8524/8524 [==============================] - 55s - loss: 0.5116 - acc: 0.7437 - val_loss: 0.4536 - val_acc: 0.7873
Epoch 4/150
8524/8524 [==============================] - 54s - loss: 0.4377 - acc: 0.8030 - val_loss: 0.4070 - val_acc: 0.8200
Epoch 5/150
8524/8524 [==============================] - 54s - loss: 0.4050 - acc: 0.8192 - val_loss: 0.3916 - val_acc: 0.8288
Epoch 6/150
8524/8524 [==============================] - 53s - loss: 0.3911 - acc: 0.8268 - val_loss: 0.3807 - val_acc: 0.8335
Epoch 7/150
8524/8524 [==============================] - 53s - loss: 0.3813 - acc: 0.8314 - val_loss: 0.4186 - val_acc: 0.7933
Epoch 8/150
8524/8524 [==============================] - 50s - 

weight list:

model_weights <- plane images first model
Test accuracy:0.98

model_weights-1 <- plane images first model dense layer 128 batch size 1000 150 epochs
30s por epoch
Test loss: 0.16379118345
Test accuracy: 0.938775510204

model_weights-2 <- plane images first model dense layer 500 batch size 1000 150 epochs
40s por epoch
Test loss: 0.123215783985
Test accuracy: 0.956977385549

model_weights-3 <- plane images first model dense layer 500 batch size a 128 150 epochs
45s per epoch
Test loss: 0.118130825795
Test accuracy: 0.975730832874

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))  # Anterior 128
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

model_weights-4 <-  with croped errors 150 epochs 1000 bach size
86s per epoch
Test loss: 0.548020246369
Test accuracy: 0.829737205987

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))  # Anterior 128
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    
model_weights-5 <- malaria (only croped) no malaria (croped and corrected) 150 epochs 128 bach size
50s per epoch  
Test loss: 0.468808364531
Test accuracy: 0.873303167421

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))  # Anterior 128
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))



Probar otro random state
# PAra hacer pooling por enmedio
model_weights-6 <- malaria (only croped) no malaria (croped and corrected) 150 epochs 128 bach size
s per epoch    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))  # Anterior 128
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
   
    
    

# Probar modificando los dropouts


